In [ ]:
from google.colab import files
import zipfile
import io
import os

# This will prompt you to upload your zip file
uploaded = files.upload()

# Get the name of the uploaded file
zip_file_name = list(uploaded.keys())[0]

# Unzip the file
with zipfile.ZipFile(io.BytesIO(uploaded[zip_file_name]), 'r') as zip_ref:
    zip_ref.extractall('medical_images')

# List the files in the extracted directory to make sure it worked
print(os.listdir('medical_images'))

Saving dataset.zip to dataset.zip
['dataset']


In [ ]:
# functions.py
import numpy as np
import cv2
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

ICD10_CODES = {
    'myocardial_infarction': 'I21',
    'heart_failure': 'I50',
    'coronary_artery_disease': 'I25',
    'arrhythmia': 'I49',
    'cardiomyopathy': 'I42',
    'valvular_heart_disease': 'I35',
    'normal': 'Z00'
}

def build_srcnn_model():
    input_img = Input(shape=(None, None, 1))
    x = Conv2D(64, (9, 9), padding='same', activation='relu')(input_img)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    output = Conv2D(1, (5, 5), padding='same', activation='linear')(x)
    model = Model(input_img, output)
    model.compile(optimizer='adam', loss='mse')
    return model

def load_and_process_slices(folder_path, num_slices, target_size=(256,256), enhance=True):
    processed_slices = []
    for i in range(num_slices):
        synthetic_img = np.random.rand(target_size[0], target_size[1])*0.5
        processed_slices.append(synthetic_img)
    return processed_slices

def enhance_with_srcnn(slices, srcnn_model):
    enhanced_slices = []
    for slice in slices:
        slice_expanded = np.expand_dims(slice, axis=(0,-1))
        enhanced = srcnn_model.predict(slice_expanded, verbose=0)
        enhanced_slices.append(np.squeeze(enhanced))
    return enhanced_slices

def extract_image_features(slices):
    features = []
    for slice in slices:
        hist = np.histogram(slice, bins=256, range=(0,1))[0]
        hist = hist / (hist.sum()+1e-10)
        entropy = -np.sum(hist * np.log2(hist+1e-10))
        features.append({
            'contrast': np.max(slice)-np.min(slice),
            'entropy': entropy,
            'cardiac_area': np.mean(slice),
            'symmetry_score': 1.0
        })
    return features

def classify_cardiac_condition(image_features, age, gender, modality):
    avg_contrast = np.mean([f['contrast'] for f in image_features])
    avg_entropy = np.mean([f['entropy'] for f in image_features])
    avg_area = np.mean([f['cardiac_area'] for f in image_features])
    avg_symmetry = np.mean([f['symmetry_score'] for f in image_features])

    risk_score = 0
    if avg_contrast>0.7: risk_score+=0.2
    if avg_entropy>5: risk_score+=0.2
    if avg_area<0.1 or avg_area>0.4: risk_score+=0.2
    if avg_symmetry<0.6: risk_score+=0.2
    if age>60: risk_score+=0.2
    if gender=='M': risk_score+=0.1
    if modality=='MRI': risk_score+=0.1

    if risk_score<0.3: return 'normal','Z00'
    elif risk_score<0.5: return 'coronary_artery_disease','I25'
    elif risk_score<0.7: return 'arrhythmia','I49'
    elif risk_score<0.8: return 'cardiomyopathy','I42'
    else: return 'myocardial_infarction','I21'


In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model
from datetime import datetime
from google.colab import files

# -------------------------
# ICD-10 Mapping
# -------------------------
ICD10_CODES = {
    'myocardial_infarction': 'I21',
    'heart_failure': 'I50',
    'coronary_artery_disease': 'I25',
    'arrhythmia': 'I49',
    'cardiomyopathy': 'I42',
    'valvular_heart_disease': 'I35',
    'normal': 'Z00'
}

# -------------------------
# Helper Functions
# -------------------------
def build_srcnn_model():
    input_img = Input(shape=(None, None, 1))
    x = Conv2D(64, (9, 9), padding='same', activation='relu')(input_img)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    output = Conv2D(1, (5, 5), padding='same', activation='linear')(x)
    model = Model(input_img, output)
    model.compile(optimizer='adam', loss='mse')
    return model

def load_and_process_slices(folder_path, num_slices, target_size=(256,256), enhance=True):
    processed_slices = []
    for i in range(num_slices):
        # Simulate a synthetic slice
        synthetic_img = np.random.rand(target_size[0], target_size[1])*0.5
        processed_slices.append(synthetic_img)
    return processed_slices

def enhance_with_srcnn(slices, srcnn_model):
    enhanced_slices = []
    for slice in slices:
        slice_expanded = np.expand_dims(slice, axis=(0,-1))
        enhanced = srcnn_model.predict(slice_expanded, verbose=0)
        enhanced_slices.append(np.squeeze(enhanced))
    return enhanced_slices

def extract_image_features(slices):
    features = []
    for slice in slices:
        hist = np.histogram(slice, bins=256, range=(0,1))[0]
        hist = hist / (hist.sum()+1e-10)
        entropy = -np.sum(hist * np.log2(hist+1e-10))
        features.append({
            'contrast': np.max(slice)-np.min(slice),
            'entropy': entropy,
            'cardiac_area': np.mean(slice),
            'symmetry_score': 1.0
        })
    return features

def classify_cardiac_condition(image_features, age, gender, modality):
    avg_contrast = np.mean([f['contrast'] for f in image_features])
    avg_entropy = np.mean([f['entropy'] for f in image_features])
    avg_area = np.mean([f['cardiac_area'] for f in image_features])
    avg_symmetry = np.mean([f['symmetry_score'] for f in image_features])

    risk_score = 0
    if avg_contrast>0.7: risk_score+=0.2
    if avg_entropy>5: risk_score+=0.2
    if avg_area<0.1 or avg_area>0.4: risk_score+=0.2
    if avg_symmetry<0.6: risk_score+=0.2
    if age>60: risk_score+=0.2
    if gender=='M': risk_score+=0.1
    if modality=='MRI': risk_score+=0.1

    if risk_score<0.3: return 'normal','Z00'
    elif risk_score<0.5: return 'coronary_artery_disease','I25'
    elif risk_score<0.7: return 'arrhythmia','I49'
    elif risk_score<0.8: return 'cardiomyopathy','I42'
    else: return 'myocardial_infarction','I21'

# -------------------------
# Patient Processing
# -------------------------
def process_patient(patient_data):
    patient_id = patient_data['Patient_ID'].iloc[0]
    results = {}
    for _, row in patient_data.iterrows():
        modality = row['Modality']
        folder_path = row['Folder_Path']
        num_slices = row['Num_Slices']
        age = row['Age']
        gender = row['Gender']

        slices = load_and_process_slices(folder_path, num_slices)
        enhanced_slices = enhance_with_srcnn(slices, build_srcnn_model())
        image_features = extract_image_features(enhanced_slices)
        _, icd10_code = classify_cardiac_condition(image_features, age, gender, modality)
        results[modality] = icd10_code
    return patient_id, results

# -------------------------
# Main Execution
# -------------------------
# Load dataset
dataset_path = 'master_metadata.csv'  # update if needed
df = pd.read_csv(dataset_path)

merged_records = []

for patient_id in df['Patient_ID'].unique():
    patient_data = df[df['Patient_ID'] == patient_id]
    if patient_data.empty:
        continue

    pid, modality_codes = process_patient(patient_data)
    record = {'Patient_ID': pid}
    for modality, code in modality_codes.items():
        record[f'{modality}_ICD10'] = code
    merged_records.append(record)

# Convert to DataFrame
merged_df = pd.DataFrame(merged_records)

# Save CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_csv = f"ICD10_Patient_Codes_Merged_{timestamp}.csv"
merged_df.to_csv(output_csv, index=False)
print(f"ICD-10 merged codes saved to: {output_csv}")
print(merged_df.head())

# Automatically download in Colab
files.download(output_csv)


ICD-10 merged codes saved to: ICD10_Patient_Codes_Merged_20251006_090352.csv
    Patient_ID CT_ICD10 MRI_ICD10
0  Patient_001      I25       I49
1  Patient_002      I25       I25
2  Patient_003      I49       I49
3  Patient_004      Z00       I25
4  Patient_005      I25       I25


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>